In [2]:
import sys
sys.path.append('/Applications/anaconda/lib/python2.7/site-packages')

In [3]:
import numpy as np
import pandas as pd
import nltk

In [65]:
train = pd.read_csv('train.csv',encoding="ISO-8859-1")
test = pd.read_csv('test.csv',encoding="ISO-8859-1")
description = pd.read_csv('product_descriptions.csv',encoding="ISO-8859-1")

In [61]:
train.head(3)

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.0
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.5
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.0


In [12]:
test.head(3)

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able


In [13]:
description.head(3)

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...


In [62]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
tokenizer = RegexpTokenizer(r'\w+')
stemmer = SnowballStemmer('english')
st = WordNetLemmatizer()

In [63]:
def generateDic(item):
    text = " ".join([st.lemmatize(stemmer.stem(word),'v') for word in tokenizer.tokenize(item.lower()) if word not in stopwords.words('english')])
    dic = dict()
    for w in text.split():
        if w in dic.keys():
            dic[w] = dic[w]+1
        else:
            dic[w] = 1
    return dic

In [66]:
train['productDict']=train['product_title'].map(lambda x: generateDic(x))

In [67]:
train['searchDict']=train['search_term'].map(lambda x: generateDic(x))

In [71]:
train.head(3)

,id,product_uid,product_title,search_term,relevance,productDict,searchDict
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.0,"{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'angl': 1, u'bracket': 1}"
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.5,"{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'bracket': 1, u'l': 1}"
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.0,"{u'tugboat': 1, u'deckov': 1, u'premium': 1, u...",{u'deck': 1}


In [68]:
test['productDict']=test['product_title'].map(lambda x: generateDic(x))

In [72]:
test['searchDict']=test['search_term'].map(lambda x: generateDic(x))

In [73]:
test.head(3)

,id,product_uid,product_title,search_term,productDict,searchDict
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket,"{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'90': 1, u'degre': 1, u'bracket': 1}"
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets,"{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'bracket': 1, u'metal': 1, u'l': 1}"
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able,"{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'sku': 1, u'abl': 1, u'simpson': 1}"


In [74]:
description['desDict']=description['product_description'].map(lambda x: generateDic(x))

In [75]:
description.head(3)

,product_uid,product_description,desDict
0,100001,"Not only do angles make joints stronger, they ...","{u'repair': 1, u'outdoor': 1, u'zmax': 1, u'ho..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...,"{u'galloncr': 1, u'sidewalks100': 1, u'composi..."
2,100003,Classic architecture meets contemporary design...,"{u'curv': 3, u'b45': 1, u'featur': 2, u'classi..."


In [79]:
train_clean = pd.merge(train,description,how='left',on='product_uid')
column_order=['id','product_uid','product_title','search_term','product_description','relevance', 'productDict','searchDict','desDict']
train_clean = train_clean[column_order]

In [80]:
train_clean.head(3)

,id,product_uid,product_title,search_term,product_description,relevance,productDict,searchDict,desDict
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,"Not only do angles make joints stronger, they ...",3.0,"{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'angl': 1, u'bracket': 1}","{u'repair': 1, u'outdoor': 1, u'zmax': 1, u'ho..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,"Not only do angles make joints stronger, they ...",2.5,"{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'bracket': 1, u'l': 1}","{u'repair': 1, u'outdoor': 1, u'zmax': 1, u'ho..."
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,BEHR Premium Textured DECKOVER is an innovativ...,3.0,"{u'tugboat': 1, u'deckov': 1, u'premium': 1, u...",{u'deck': 1},"{u'galloncr': 1, u'sidewalks100': 1, u'composi..."


In [81]:
test_clean = pd.merge(test,description,how='left',on='product_uid')
column_order=['id','product_uid','product_title','search_term','product_description','productDict','searchDict','desDict']
test_clean = test_clean[column_order]

In [82]:
test_clean.head(3)

,id,product_uid,product_title,search_term,product_description,productDict,searchDict,desDict
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket,"Not only do angles make joints stronger, they ...","{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'90': 1, u'degre': 1, u'bracket': 1}","{u'repair': 1, u'outdoor': 1, u'zmax': 1, u'ho..."
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets,"Not only do angles make joints stronger, they ...","{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'bracket': 1, u'metal': 1, u'l': 1}","{u'repair': 1, u'outdoor': 1, u'zmax': 1, u'ho..."
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able,"Not only do angles make joints stronger, they ...","{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'sku': 1, u'abl': 1, u'simpson': 1}","{u'repair': 1, u'outdoor': 1, u'zmax': 1, u'ho..."


In [ ]:
train_clean.to_csv('train_clean.csv',index=False,encoding="ISO-8859-1")
test_clean.to_csv('test_clean.csv',index=False,encoding="ISO-8859-1")